In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from mpl_toolkits.mplot3d import Axes3D
from sklearn import linear_model

# Population Data Column preprocessing

In [3]:
# data call
df_pop = pd.read_csv('../다중회귀분석/행정동별_인구수_원본.csv', index_col=False)
df_code = pd.read_csv('../다중회귀분석/행정동코드.csv', index_col=False)

In [5]:
df_code.columns

Index(['통계청행정동코드', '행자부행정동코드', '시도명', '시군구명', '행정동명'], dtype='object')

In [4]:
df_pop.columns

Index(['기준일ID', '시간대구분', '행정동코드', '총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수',
       '남자15세부터19세생활인구수', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수',
       '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '남자40세부터44세생활인구수',
       '남자45세부터49세생활인구수', '남자50세부터54세생활인구수', '남자55세부터59세생활인구수',
       '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수', '여자0세부터9세생활인구수',
       '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수',
       '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
       '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수',
       '여자55세부터59세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수',
       '여자70세이상생활인구수'],
      dtype='object')

In [6]:
# delete first trash record 
df_code = df_code.iloc[1:]

# type matching for joining with code
df_code['행자부행정동코드'] = df_code['행자부행정동코드'].astype('int64')
df_pop['행정동코드'] = df_pop['행정동코드'].astype('int64')

In [7]:
df_pop

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20231001,0,11380530,22734.2529,360.9099,224.8362,552.1779,598.9296,882.5695,836.1706,...,811.8566,862.1507,873.9038,667.0151,1019.9621,960.8058,1075.1081,1002.6932,762.4476,2039.5830
1,20231001,0,11380631,24576.9924,723.6289,397.3605,699.1521,623.6417,922.5469,817.1459,...,882.1717,927.3364,1005.0317,893.7216,1086.5414,1096.7991,1067.6682,1057.3139,710.7672,1851.1231
2,20231001,0,11140550,9048.8135,541.6812,165.5135,152.9228,390.9228,454.5985,385.9667,...,384.1645,297.1103,374.7113,399.9091,368.0235,307.4670,208.6892,136.8860,132.3833,351.7992
3,20231001,0,11350595,39636.1446,1604.7508,791.9107,1101.2529,1532.8742,1554.1134,1296.7465,...,1424.8983,1313.0002,1459.8118,1387.5974,1475.8228,1340.9569,1648.9524,1492.2480,1298.7019,3098.5651
4,20231001,0,11470540,18560.2696,648.7150,547.4209,965.5009,544.2008,626.6250,497.4201,...,592.8334,502.8279,614.1144,637.1190,1082.4648,831.0457,803.1677,574.2480,437.7856,1088.3779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315451,20231031,23,11350560,23328.3245,723.3146,372.0371,905.6096,2148.6470,1095.6983,698.0627,...,705.3973,672.4282,793.7318,755.7376,909.3189,689.4435,773.8896,749.0588,573.7800,1694.6262
315452,20231031,23,11650570,19627.1808,1194.3273,806.4547,895.6581,361.9207,326.0215,338.9087,...,369.4881,426.8944,825.1702,1075.5930,1122.3634,715.4737,641.2286,423.3113,401.6018,1097.1644
315453,20231031,23,11680630,30933.8632,780.3734,855.8412,2173.2886,915.2354,1101.5112,1176.4238,...,1212.6021,1148.9860,1311.5816,1414.4604,1862.3325,1245.7462,886.7439,632.8622,441.0081,1025.4172
315454,20231031,23,11290770,12949.8318,368.1497,256.7063,350.8151,368.1697,451.8408,430.3816,...,525.8059,513.0067,495.9601,469.5965,485.6948,535.5495,581.4097,495.3314,468.5594,1203.8335


In [8]:
df_code

,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
1,1101053,11110530,서울,종로구,사직동
2,1101054,11110540,서울,종로구,삼청동
3,1101055,11110550,서울,종로구,부암동
4,1101056,11110560,서울,종로구,평창동
5,1101057,11110570,서울,종로구,무악동
...,...,...,...,...,...
420,1125070,11740690,서울,강동구,둔촌1동
421,1125071,11740700,서울,강동구,둔촌2동
422,1125072,11740570,서울,강동구,암사1동
423,1125073,11740610,서울,강동구,천호2동


In [9]:
merged_df = pd.merge(df_pop, df_code, left_on='행정동코드', right_on='행자부행정동코드', how='inner')

In [10]:
merged_df.columns

Index(['기준일ID', '시간대구분', '행정동코드', '총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수',
       '남자15세부터19세생활인구수', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수',
       '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '남자40세부터44세생활인구수',
       '남자45세부터49세생활인구수', '남자50세부터54세생활인구수', '남자55세부터59세생활인구수',
       '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수', '여자0세부터9세생활인구수',
       '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수',
       '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
       '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수',
       '여자55세부터59세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수',
       '통계청행정동코드', '행자부행정동코드', '시도명', '시군구명', '행정동명'],
      dtype='object')

In [11]:
# merged_df = merged_df[["기준일ID","총생활인구수","남자70세이상생활인구수","여자70세이상생활인구수","행자부행정동코드","시도명","시군구명","행정동명"]]
merged_df = merged_df[["총생활인구수","남자70세이상생활인구수","여자70세이상생활인구수","시도명","시군구명","행정동명"]]


In [12]:
merged_df

,총생활인구수,남자70세이상생활인구수,여자70세이상생활인구수,시도명,시군구명,행정동명
0,22734.2529,1257.6644,2039.5830,서울,은평구,불광2동
1,23030.3085,1271.7907,2072.7906,서울,은평구,불광2동
2,23089.1904,1267.5384,2084.4019,서울,은평구,불광2동
3,22988.7409,1245.8931,2068.4004,서울,은평구,불광2동
4,23072.1011,1242.0892,2068.1557,서울,은평구,불광2동
...,...,...,...,...,...,...
315451,5951.1916,226.3781,319.1454,서울,종로구,가회동
315452,5181.8576,210.7768,314.9813,서울,종로구,가회동
315453,4576.6548,181.6859,314.5297,서울,종로구,가회동
315454,4210.6528,174.3895,336.9003,서울,종로구,가회동


# Center data column preprocessing

In [50]:
# data call
df_senior_center = pd.read_csv('./다중회귀분석/경로당정보.csv', index_col=False)

# df_leisure_center = pd.read_csv('./다중회귀분석/여가복지시설정보.csv', index_col=False)
# 레저 센터를 제외한 이유 : 레코드가 99밖에 없기도하고, 주소 단위가 동단위가 아닌 구단위여서 사용하기 어렵다 판단.

In [51]:
df_senior_center = df_senior_center[['소재지전체주소', '상세영업상태명', '입소정원']]

In [52]:
df_senior_center

,소재지전체주소,상세영업상태명,입소정원
0,서울특별시 은평구 응암동 백련산힐스테이트2차아파트,운영,NaN
1,서울특별시 은평구 응암동,운영,0.0
2,서울특별시 영등포구 양평동3가 36번지,운영,0.0
3,서울특별시 마포구 공덕동,운영,0.0
4,서울특별시 강서구 마곡동,운영,58.0
...,...,...,...
3740,서울특별시 마포구 상암동 상암월드컵파크11단지,운영,36.0
3741,서울특별시 마포구 상암동 상암월드컵파크12단지,운영,NaN
3742,서울특별시 마포구 아현동 104동,운영,23.0
3743,서울특별시 은평구 진관동 은평뉴타운구파발,운영,22.0


In [53]:
# 동 정보가 없는 데이터 삭제 (1개 삭제됨 ㅎ)
df_senior_center = df_senior_center.drop(df_senior_center[df_senior_center['소재지전체주소'].apply(lambda x: len(x.split(" ")) < 3 )].index)
df_senior_center.fillna(0, inplace=True)
average_capacity = df_senior_center['입소정원'].mean()
df_senior_center['입소정원'].replace(0.0, average_capacity, inplace=True)
df_senior_center['입소정원'] = df_senior_center['입소정원'].astype(int)
# 동 정보로 데이터 변경
df_senior_center['소재지전체주소'] = df_senior_center['소재지전체주소'].apply(lambda x: x.split(" ")[2])

In [54]:
df_senior_center

,소재지전체주소,상세영업상태명,입소정원
0,응암동,운영,29
1,응암동,운영,29
2,양평동3가,운영,29
3,공덕동,운영,29
4,마곡동,운영,58
...,...,...,...
3740,상암동,운영,36
3741,상암동,운영,29
3742,아현동,운영,23
3743,진관동,운영,22


In [75]:
grouped_df = df_senior_center.groupby('소재지전체주소').agg({'상세영업상태명': 'first', '입소정원': 'min', '상세영업상태명': 'size'}).reset_index()
grouped_df.rename(columns={'상세영업상태명': '총노인시설수'}, inplace=True)
grouped_df

,소재지전체주소,총노인시설수,입소정원
0,가락동,24,20
1,가리봉동,4,29
2,가산동,10,20
3,가양동,29,21
4,가회동,1,29
...,...,...,...
321,효제동,2,29
322,효창동,4,35
323,후암동,5,38
324,휘경동,18,3


# For clustering, Data Joining

In [88]:
import random

refind_df = pd.merge(merged_df, grouped_df, left_on='행정동명', right_on='소재지전체주소', how='left')
refind_df['총노인시설수'].fillna(0, inplace=True)

refind_df.drop("소재지전체주소", axis=1, inplace=True)

result_df = refind_df.groupby(['시도명', '시군구명', '행정동명']).mean().reset_index()


result_df.fillna(average_capacity, inplace=True)

result_df['입소정원'] = result_df['입소정원'].astype(int)
result_df

,시도명,시군구명,행정동명,총생활인구수,남자70세이상생활인구수,여자70세이상생활인구수,총노인시설수,입소정원
0,서울,강남구,개포1동,8277.465083,355.637820,542.953295,0.0,29
1,서울,강남구,개포2동,33725.943141,1294.313774,2297.302731,0.0,29
2,서울,강남구,개포4동,16339.446267,543.909089,896.641961,0.0,29
3,서울,강남구,논현1동,40786.363715,977.736934,1542.546077,0.0,29
4,서울,강남구,논현2동,44684.994820,1186.884946,1794.271896,0.0,29
...,...,...,...,...,...,...,...,...
419,서울,중랑구,상봉2동,25041.936681,939.899016,1584.525416,0.0,29
420,서울,중랑구,신내1동,33076.724608,1459.433770,2481.588866,0.0,29
421,서울,중랑구,신내2동,17158.080798,1020.204181,1476.938172,0.0,29
422,서울,중랑구,중화1동,14467.961694,686.151036,1120.033997,0.0,29


# Export as CSV

In [89]:
result_df.to_csv('./다중회귀분석/행정동별_인구수_노인시설수.csv', index=False)